### Çoklu Doğrusal Regresyonda Tahmin Başarısı

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score

df = pd.read_csv('datasets/advertising.csv')
df

,TV,radio,newspaper,sales
0,230.10,37.80,69.20,22.10
1,44.50,39.30,45.10,10.40
2,17.20,45.90,69.30,9.30
3,151.50,41.30,58.50,18.50
4,180.80,10.80,58.40,12.90
...,...,...,...,...
195,38.20,3.70,13.80,7.60
196,94.20,4.90,8.10,9.70
197,177.00,9.30,6.40,12.80
198,283.60,42.00,66.20,25.50


In [2]:
# Bağımsız değişkenlerin seçimi:
x = df.drop('sales', axis=1)

# Bağımlı değişken seçimi:
y = df[["sales"]]

###### Model

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)
reg_model = LinearRegression().fit(x_train, y_train)

###### Tahmin

In [4]:
# Bu ifadeyi fonksiyonel şekilde yazmak için:

# Liste oluşturalım:
yeni_veri = [[30], [10], [40]]

# Bunu bir dataframe'e çevirelim:
yeni_veri = pd.DataFrame(yeni_veri).T
yeni_veri

,0,1,2
0,30,10,40


In [5]:
reg_model.predict(yeni_veri)

C:\Users\aleyy\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[6.202131]])

### Tahmin Başarısını Değerlendirme

In [12]:
# HOLDOUT YÖNTEMİ

In [7]:
# Train RMSE
y_pred = reg_model.predict(x_train)
np.sqrt(mean_squared_error(y_train, y_pred))

1.736902590147092

In [8]:
# Train R-KARE
reg_model.score(x_train, y_train)

0.8959372632325174

In [9]:
# Eskiden %60'larda olan değer şimdi %90'a geldi. Demek ki underfit etmiştik. İyi açıklayamıyorduk. Yeni değişkenler 
# eklendiğinde daha iyi açıklayabildik ve hata oranımız da azaldı.

In [10]:
# Test RMSE
y_pred = reg_model.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred))

1.411341755858159

In [11]:
# Test R-KARE
reg_model.score(x_test, y_test)

0.8927605914615383

* Normalde test hatası train hatasından daha yüksek çıkar. Dolayısıyla bu küçük dataset için çok güzel bir senaryo yani oldukça iyi bir durum. Beklenti dışı ve güzel bir durum.

In [13]:
# CROSSVALIDATION YÖNTEMİ

In [15]:
# Çapraz doğrulamayı veri seti az veriden oluştuğundan bir ayrım yapmadan tüm veriyle gerçekleştirdik.

# 10 Katlı CV RMSE
np.mean(np.sqrt(-cross_val_score(reg_model, x, y, cv=10, scoring="neg_mean_squared_error")))

1.6913531708051797

* neg_mean_squared_error negatif ortalama hatayı veriyor bu sebeple bunu eksiyle çarptık.

In [17]:
# Eksiyle çarpmazsak:
cross_val_score(reg_model, x, y, cv=10, scoring="neg_mean_squared_error")

array([-3.56038438, -3.29767522, -2.08943356, -2.82474283, -1.3027754 ,
       -1.74163618, -8.17338214, -2.11409746, -3.04273109, -2.45281793])

In [18]:
# Eksiyle çarparsak:
-cross_val_score(reg_model, x, y, cv=10, scoring="neg_mean_squared_error")

array([3.56038438, 3.29767522, 2.08943356, 2.82474283, 1.3027754 ,
       1.74163618, 8.17338214, 2.11409746, 3.04273109, 2.45281793])

In [19]:
np.sqrt(-cross_val_score(reg_model, x, y, cv=10, scoring="neg_mean_squared_error"))

array([1.88689808, 1.81595022, 1.44548731, 1.68069713, 1.14139187,
       1.31971064, 2.85891276, 1.45399362, 1.7443426 , 1.56614748])

* Bu sonuçlardan hangisine güvenmeliyiz?
* Veri seti sayımız çok olsaydı herhangi birini seçmek yanlış olmayabilirdi. Ancak veri seti sayımız az olduğundan dolayı 100 Katlı öapraz doğrulama yöntemine daha çok güvenmek iyi olabilir. Veri setinin boyutu az olduğu için de 10 parça yerine 5 parça mı yapsak yorumu da geçerli 